In [1]:
import pandas as pd
import numpy as np

In [2]:
clinical_df = pd.read_csv('data/clinical.txt',sep='\t').set_index('sampleID')
clinical_columns = [col for col in clinical_df.columns if not col.startswith('_')]
clinical_df = clinical_df.loc[:,clinical_columns]
gene_df = pd.read_csv('data/gene_expression_array.txt',sep='\t').set_index('sample').T
original_gene_df = pd.read_csv('data/gene_expression_array.txt',sep='\t').set_index('sample').T
original_gene_df.index.name = 'sampleID'
# gene_df = gene_df - gene_df.mean(axis=0)
normal_samples = clinical_df.loc[clinical_df['sample_type'] == 'Solid Tissue Normal'].index.intersection(gene_df.index)
gene_df.index.name = 'sampleID'
normal_gene_df = gene_df.loc[normal_samples]
tumor_gene_df = gene_df[gene_df.index.map(lambda x:x not in normal_samples)]

In [3]:
# rnaseq_gene_df = pd.read_csv('data/UNC_gene_expression.txt',sep='\t').set_index('sample').T

# normal_rna_samples = clinical_df.loc[clinical_df['sample_type'] == 'Solid Tissue Normal'].index.intersection(rnaseq_gene_df.index)
# normal_rna_gene_df = rnaseq_gene_df.loc[rnaseq_gene_df.index.map(lambda x:x in normal_rna_samples)]
# tumor_rna_gene_df = rnaseq_gene_df.loc[rnaseq_gene_df.index.map(lambda x:x not in normal_rna_samples)]

In [4]:
clinical_df.index = clinical_df.index.map(lambda x:'-'.join(x.split('-')[:-1]))
tumor_gene_df.index = tumor_gene_df.index.map(lambda x:'-'.join(x.split('-')[:-1]))
original_gene_df.index = original_gene_df.index.map(lambda x:'-'.join(x.split('-')[:-1]))
patient_id_with_image = pd.read_csv('patient_id',header=None)[0]
intersected_index = tumor_gene_df.index.intersection(patient_id_with_image)
all_df = clinical_df[['clinical_stage']].loc[intersected_index].join(tumor_gene_df,how='inner').dropna()

In [5]:
# rna_seq_df = pd.read_csv('data/TCGA-OV.htseq_fpkm.tsv',sep='\t').set_index('Ensembl_ID').T
# rna_seq_df.index.name = 'sampleID'
# rna_seq_df.index = rna_seq_df.index.map(lambda x:'-'.join(x.split('-')[:-1]))

In [6]:
import scipy.stats as stats

In [7]:
rows = []
for gene in gene_df.columns:
    normal = normal_gene_df[gene]
    tumor = all_df[gene]
    log2fc = np.mean(tumor) - np.mean(normal)
    fc = 2**log2fc
    res = stats.ttest_ind(normal, tumor)
    rows.append([gene,res.pvalue,log2fc,fc])

In [52]:
diff_expr_df = pd.DataFrame(rows,columns =['Gene','pvalue','log2FC','FC'])
# diff_expr_df = diff_expr_df.sort_values(by='pvalue')
diff_expr_df = diff_expr_df.sort_values(by='pvalue')
diff_expr_df = diff_expr_df.reset_index().drop(columns='index')
diff_expr_df['Rank'] = diff_expr_df.index + 1
singificant_gene = diff_expr_df[diff_expr_df['pvalue'] < 0.05]
diff_expr_df['FDR_Q_value'] = diff_expr_df['Rank'] /singificant_gene.shape[0]
diff_expr_df.loc[diff_expr_df['FDR_Q_value'] > 1,'FDR_Q_value'] = 1

In [55]:
diff_expr_df[diff_expr_df['FDR_Q_value'] < 0.05]

,Gene,pvalue,log2FC,FC,Rank,FDR_Q_value
0,CCL15,2.833661e-67,-2.945431,0.129819,1,0.000193
1,DNAH9,4.713203e-67,-3.109152,0.115892,2,0.000387
2,C6,5.476052e-59,-5.965778,0.016000,3,0.000580
3,FGF14,1.943496e-56,-2.139676,0.226931,4,0.000773
4,CRISP2,1.609588e-49,-2.960001,0.128514,5,0.000966
...,...,...,...,...,...,...
253,FANCI,2.616308e-11,2.264512,4.804920,254,0.049092
254,PSRC1,2.939612e-11,2.238293,4.718383,255,0.049285
255,PSMA7,3.140950e-11,1.321216,2.498766,256,0.049478
256,MB,3.148771e-11,-1.698891,0.308023,257,0.049671


In [ ]:
&(diff_expr_df['pvalue'] < 0.05)

In [97]:
gene_list = diff_expr_df[(diff_expr_df['FDR_Q_value']<0.05)  & (np.abs(diff_expr_df['log2FC']) > np.log2(2))]

In [10]:
from bioinfokit import visuz

In [98]:
diff_expr_df

,Gene,pvalue,log2FC,FC,Rank,FDR_Q_value
0,CCL15,2.833661e-67,-2.945431,0.129819,1,0.000193
1,DNAH9,4.713203e-67,-3.109152,0.115892,2,0.000387
2,C6,5.476052e-59,-5.965778,0.016000,3,0.000580
3,FGF14,1.943496e-56,-2.139676,0.226931,4,0.000773
4,CRISP2,1.609588e-49,-2.960001,0.128514,5,0.000966
...,...,...,...,...,...,...
12037,SOCS7,9.994309e-01,0.000049,1.000034,12038,1.000000
12038,CD52,9.998415e-01,0.000076,1.000053,12039,1.000000
12039,PYHIN1,9.998427e-01,0.000017,1.000012,12040,1.000000
12040,ABLIM1,9.999217e-01,-0.000031,0.999978,12041,1.000000


In [ ]:
# list(upregulated_gene['Gene'])+list(downregulated_gene['Gene'])

In [85]:
diff_expr_df

,Gene,pvalue,log2FC,FC,Rank,FDR_Q_value
0,CCL15,2.833661e-67,-2.945431,0.129819,1,0.000193
1,DNAH9,4.713203e-67,-3.109152,0.115892,2,0.000387
2,C6,5.476052e-59,-5.965778,0.016000,3,0.000580
3,FGF14,1.943496e-56,-2.139676,0.226931,4,0.000773
4,CRISP2,1.609588e-49,-2.960001,0.128514,5,0.000966
...,...,...,...,...,...,...
12037,SOCS7,9.994309e-01,0.000049,1.000034,12038,1.000000
12038,CD52,9.998415e-01,0.000076,1.000053,12039,1.000000
12039,PYHIN1,9.998427e-01,0.000017,1.000012,12040,1.000000
12040,ABLIM1,9.999217e-01,-0.000031,0.999978,12041,1.000000


In [99]:
upregulated_gene = gene_list[gene_list['log2FC'] > 0]
downregulated_gene = gene_list[gene_list['log2FC'] < 0]

In [100]:
upregulated_gene['Gene'].to_csv('upregulated_gene_list.txt',index=False,header=False)
downregulated_gene['Gene'].to_csv('downregulated_gene_list.txt',index=False,header=False)

In [105]:
visuz.GeneExpression.volcano(df=diff_expr_df,sign_line=True,legendpos='upper right', color=("#00239CFF", "grey", "#E10600FF"),lfc_thr=(1.0,1.0),pv_thr=(0.05,0.05),
                             axxlabel='log2(FC)',legendanchor=(1.5,1),plotlegend=True, lfc='log2FC',pv='pvalue',geneid='Gene',gfont=7,
                             genenames=('C6'), legendlabels=['Up-regulated','Not significant','Down-regulated'])

In [143]:
# all_df[['clinical_stage','BRCA2']].to_csv('BRCA2_expression.tsv',sep='\t')

In [104]:
all_df[['clinical_stage']+list(gene_list['Gene'])].to_csv('all_DEG_expression.tsv',sep='\t')

In [13]:
# all_df[['clinical_stage','BRCA2']].join(original_gene_df['BRCA2'],how='inner',lsuffix='_zero_centered',rsuffix='_original').to_csv('BRCA2_expression.tsv',sep='\t')

In [162]:
# all_df[['clinical_stage']+list(gene_list['Gene'])].to_csv('all_DEG_expression.tsv',sep='\t')

In [163]:
upregulated_gene

,Gene,pvalue,log2FC,FC
32,SP3,2.195536e-09,1.095140,2.136337
45,GTSE1,1.061828e-07,1.011412,2.015883
88,COL4A1,6.561429e-04,1.214825,2.321126
91,CHST2,1.397521e-03,1.380494,2.603575
121,CKS1B,2.052154e-05,1.309508,2.478569
...,...,...,...,...
11992,VCAN,1.326839e-02,1.633438,3.102514
12004,UPK3B,1.804004e-02,1.125797,2.182220
12019,PLAU,4.216808e-03,1.465759,2.762087
12025,DNM1L,4.151338e-05,1.111686,2.160980


In [164]:
downregulated_gene

,Gene,pvalue,log2FC,FC
8,NDP,1.893180e-06,-2.814974,0.142105
10,PMM1,1.295277e-06,-1.051915,0.482328
76,HRH1,1.773611e-04,-1.297620,0.406797
93,CHST4,8.411821e-09,-2.299458,0.203139
137,PIPOX,1.386023e-02,-1.058359,0.480178
...,...,...,...,...
11879,ENPP2,4.291194e-09,-2.387291,0.191141
11957,C11orf75,4.064978e-06,-1.690874,0.309739
11968,TCEAL1,6.247516e-07,-1.441568,0.368167
12010,NDN,2.659224e-04,-1.804669,0.286247
